pca_glove_100features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/glove_withstopwordchangelevels_100_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,12.265925,7.578888,-12.519294,-8.052782,-3.112948,5.063536,13.978667,-9.588995,-1.777143,...,0.707909,-0.426996,-0.283272,-0.143021,0.395399,-0.670108,-0.178054,0.121648,-0.348253,-0.037353
1,1,-0.355357,-3.948469,-3.632458,0.013224,-2.745197,0.425346,3.742328,-0.000833,-4.838810,...,-0.047416,-0.217576,0.325500,0.229315,0.449607,0.137050,0.785544,0.805884,0.213045,-0.673062
2,2,-1.076368,-0.470055,-2.688537,4.352805,0.317775,-2.155223,3.692931,-0.539706,-2.631295,...,0.260720,0.566488,0.154656,-0.075269,-0.176377,-0.246283,0.213147,1.084711,0.410820,0.130875
3,3,-0.071383,-4.485360,2.669079,2.272665,-6.200417,-1.945054,-4.203540,-2.323606,-1.418835,...,0.513332,-0.357071,0.038757,0.205337,-0.018463,0.149424,0.007802,0.149985,0.198851,-0.232869
4,4,-5.559208,-2.317947,-0.473219,0.380595,-3.397183,-0.891851,-2.080894,-0.314549,0.231365,...,0.067453,-0.702117,-0.116943,-0.419481,0.057452,0.773223,0.811676,-0.418951,-0.437150,0.387109


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,12.265925,7.578888,-12.519294,-8.052782,-3.112948,5.063536,13.978667,-9.588995,-1.777143,5.193982,...,0.707909,-0.426996,-0.283272,-0.143021,0.395399,-0.670108,-0.178054,0.121648,-0.348253,-0.037353
1,-0.355357,-3.948469,-3.632458,0.013224,-2.745197,0.425346,3.742328,-0.000833,-4.838810,-3.054734,...,-0.047416,-0.217576,0.325500,0.229315,0.449607,0.137050,0.785544,0.805884,0.213045,-0.673062
2,-1.076368,-0.470055,-2.688537,4.352805,0.317775,-2.155223,3.692931,-0.539706,-2.631295,-1.414110,...,0.260720,0.566488,0.154656,-0.075269,-0.176377,-0.246283,0.213147,1.084711,0.410820,0.130875
3,-0.071383,-4.485360,2.669079,2.272665,-6.200417,-1.945054,-4.203540,-2.323606,-1.418835,-1.178769,...,0.513332,-0.357071,0.038757,0.205337,-0.018463,0.149424,0.007802,0.149985,0.198851,-0.232869
4,-5.559208,-2.317947,-0.473219,0.380595,-3.397183,-0.891851,-2.080894,-0.314549,0.231365,0.637205,...,0.067453,-0.702117,-0.116943,-0.419481,0.057452,0.773223,0.811676,-0.418951,-0.437150,0.387109


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 100), (524, 100))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.36875349130305163


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.431967,0.431967,0.496462,0.418464,0.303435,0.303435,0.355696,0.301514


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9751    0.9778    0.9765       361
           2     0.9853    0.9729    0.9791       553
           3     0.9519    0.9706    0.9612       306

    accuracy                         0.9738      1220
   macro avg     0.9708    0.9738    0.9722      1220
weighted avg     0.9739    0.9738    0.9738      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2667    0.1019    0.1475       157
           2     0.4411    0.8059    0.5701       237
           3     0.3226    0.0769    0.1242       130

    accuracy                         0.4141       524
   macro avg     0.3435    0.3282    0.2806       524
weighted avg     0.3594    0.4141    0.3329       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.431967,0.431967,0.496462,0.418464,0.303435,0.303435,0.355696,0.301514
1,RandomForest,0.973770,0.973770,0.973943,0.973816,0.414122,0.414122,0.359437,0.332875


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5379    0.4127    0.4671       361
           2     0.5345    0.7143    0.6115       553
           3     0.5686    0.3791    0.4549       306

    accuracy                         0.5410      1220
   macro avg     0.5470    0.5020    0.5111      1220
weighted avg     0.5441    0.5410    0.5295      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2982    0.2166    0.2509       157
           2     0.4620    0.6160    0.5280       237
           3     0.2553    0.1846    0.2143       130

    accuracy                         0.3893       524
   macro avg     0.3385    0.3391    0.3311       524
weighted avg     0.3617    0.3893    0.3672       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.513115,0.513115,0.525180,0.490582,0.393130,0.393130,0.358552,0.354499
1,RandomForest,0.973770,0.973770,0.973785,0.973720,0.410305,0.410305,0.345921,0.342000
2,Adaboost,0.540984,0.540984,0.544070,0.529469,0.389313,0.389313,0.361672,0.367166


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9783    0.8753    0.9240       361
           2     0.8805    0.9855    0.9300       553
           3     0.9604    0.8725    0.9144       306

    accuracy                         0.9246      1220
   macro avg     0.9397    0.9111    0.9228      1220
weighted avg     0.9295    0.9246    0.9243      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2900    0.1847    0.2257       157
           2     0.4426    0.6835    0.5373       237
           3     0.2931    0.1308    0.1809       130

    accuracy                         0.3969       524
   macro avg     0.3419    0.3330    0.3146       524
weighted avg     0.3598    0.3969    0.3555       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.513115,0.513115,0.525180,0.490582,0.393130,0.393130,0.358552,0.354499
1,RandomForest,0.973770,0.973770,0.973785,0.973720,0.410305,0.410305,0.345921,0.342000
2,Adaboost,0.540984,0.540984,0.544070,0.529469,0.389313,0.389313,0.361672,0.367166
3,GBM,0.924590,0.924590,0.929474,0.924316,0.396947,0.396947,0.359800,0.355507


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4436    0.4903    0.4658       361
           2     0.5880    0.4774    0.5269       553
           3     0.4409    0.5359    0.4838       306

    accuracy                         0.4959      1220
   macro avg     0.4908    0.5012    0.4922      1220
weighted avg     0.5084    0.4959    0.4980      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3200    0.3567    0.3373       157
           2     0.4722    0.4304    0.4503       237
           3     0.3083    0.3154    0.3118       130

    accuracy                         0.3798       524
   macro avg     0.3668    0.3675    0.3665       524
weighted avg     0.3859    0.3798    0.3821       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.431967,0.431967,0.496462,0.418464,0.303435,0.303435,0.355696,0.301514
1,RandomForest,0.973770,0.973770,0.973943,0.973816,0.414122,0.414122,0.359437,0.332875
2,SVM,0.495902,0.495902,0.508357,0.498022,0.379771,0.379771,0.385939,0.382108


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5398    0.5069    0.5229       361
           2     0.5678    0.7577    0.6491       553
           3     0.5105    0.2386    0.3252       306

    accuracy                         0.5533      1220
   macro avg     0.5394    0.5011    0.4990      1220
weighted avg     0.5451    0.5533    0.5305      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3375    0.3439    0.3407       157
           2     0.4934    0.6329    0.5545       237
           3     0.3667    0.1692    0.2316       130

    accuracy                         0.4313       524
   macro avg     0.3992    0.3820    0.3756       524
weighted avg     0.4153    0.4313    0.4103       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.513115,0.513115,0.525180,0.490582,0.393130,0.393130,0.358552,0.354499
1,RandomForest,0.973770,0.973770,0.973785,0.973720,0.410305,0.410305,0.345921,0.342000
2,Adaboost,0.540984,0.540984,0.544070,0.529469,0.389313,0.389313,0.361672,0.367166
3,GBM,0.924590,0.924590,0.929474,0.924316,0.396947,0.396947,0.359800,0.355507
4,SVM,0.545082,0.545082,0.563311,0.504611,0.417939,0.417939,0.378629,0.366317
5,KNN,0.553279,0.553279,0.545125,0.530500,0.431298,0.431298,0.415258,0.410339


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4169    0.3546    0.3832       361
           2     0.5115    0.7269    0.6004       553
           3     0.5512    0.2288    0.3233       306

    accuracy                         0.4918      1220
   macro avg     0.4932    0.4368    0.4357      1220
weighted avg     0.4934    0.4918    0.4667      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3382    0.2930    0.3140       157
           2     0.4575    0.6582    0.5398       237
           3     0.4468    0.1615    0.2373       130

    accuracy                         0.4256       524
   macro avg     0.4142    0.3709    0.3637       524
weighted avg     0.4191    0.4256    0.3971       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.513115,0.513115,0.525180,0.490582,0.393130,0.393130,0.358552,0.354499
1,RandomForest,0.973770,0.973770,0.973785,0.973720,0.410305,0.410305,0.345921,0.342000
2,Adaboost,0.540984,0.540984,0.544070,0.529469,0.389313,0.389313,0.361672,0.367166
3,GBM,0.924590,0.924590,0.929474,0.924316,0.396947,0.396947,0.359800,0.355507
4,SVM,0.545082,0.545082,0.563311,0.504611,0.417939,0.417939,0.378629,0.366317
5,KNN,0.553279,0.553279,0.545125,0.530500,0.431298,0.431298,0.415258,0.410339
6,GNB,0.491803,0.491803,0.493449,0.466666,0.425573,0.425573,0.419104,0.397090
